In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct  5 10:19:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # prevent from sleeping
# import requests
# import time

# while True:
#     try:
#         requests.get('https://www.google.com')
#         print("Kept alive.")
#     except:
#         print("Failed to keep alive.")
#     time.sleep(600)

In [ ]:
# # increase available memory
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

In [ ]:
%cd drive/MyDrive/cits3001_project/reinforcement-learning

[Errno 2] No such file or directory: 'drive/MyDrive/cits3001_project/reinforcement-learning'
/content/drive/MyDrive/cits3001_project/reinforcement-learning


In [ ]:
%pwd

'/content/drive/My Drive/cits3001_project/reinforcement-learning'

In [ ]:
!sudo apt-get install python3.7
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3
!sudo apt install python3-pip
!sudo apt install freeglut3-dev
!sudo apt install python3.7-distutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.7 is already the newest version (3.7.17-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.7
Nothing to configure.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.7-distutils is already the newest version (3.7.17-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and

In [ ]:
!python --version

Python 3.7.17


In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py
!pip install gym==v0.21.0
!pip install stable-baselines3==1.5.0
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [ ]:
!pip show gym_super_mario_bros

Name: gym-super-mario-bros
Version: 7.3.0
Summary: Super Mario Bros. for OpenAI Gym
Home-page: https://github.com/Kautenja/gym-super-mario-bros
Author: Christian Kauten
Author-email: kautencreations@gmail.com
License: Proprietary
Location: /usr/local/lib/python3.7/dist-packages
Requires: nes-py
Required-by: 


In [ ]:
# !pip install wrappers

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/dist-packages')
# #import the game
import gym
import gym_super_mario_bros
import os
# #import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# #import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
# from gym.wrappers import SkipFrame, ResizeObservation
from matplotlib import pyplot as plt

In [ ]:
####################################################################
# Set up game environment and simplify actions
env = gym_super_mario_bros.make("SuperMarioBros-1-2-v0")
# sample the ['right'] and ['right', 'A'] actions
# MY_MOVEMENT = SIMPLE_MOVEMENT[1:3]
# env = JoypadSpace(env, MY_MOVEMENT)
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Skip frames
# env = SkipFrame(env, skip=4)

# Grayscale environment
env = GrayScaleObservation(env, keep_dim=True)

# Resize observation
# env = ResizeObservation(env, shape=84)

# Create a vectorized wrapper for an environment
env = DummyVecEnv([lambda: env])

# # Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
###################################################################
# Train and save the training process
###################################################################
class TrainAndSaveCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndSaveCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(
                self.save_path, '1-2_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True


DATA_DIR = './train/'
LOG_DIR = './logs/'

callback = TrainAndSaveCallback(check_freq=50000, save_path=DATA_DIR)
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR,
            learning_rate=0.000005, n_steps=512)
model.learn(4000000, callback=callback)

model.save('1-2-model')

Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to ./logs/PPO_7
----------------------------
| time/              |     |
|    fps             | 52  |
|    iterations      | 1   |
|    time_elapsed    | 9   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 10            |
|    iterations           | 2             |
|    time_elapsed         | 101           |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 5.4121483e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.000316     |
|    learning_rate        | 1e-06         |
|    loss                 | 327           |
|    n_updates            | 10            |
|    policy_gradient_loss | -2.26e-05     |
|    value_

In [ ]:
#####################################################################
# Load the training result
#####################################################################
# from IPython.display import clear_output, display
# from PIL import Image
# import numpy as np

# model = PPO.load('./train/1-2_model_50000')
# state = env.reset()
# while True:
#     action, _ = model.predict(state)
#     print(SIMPLE_MOVEMENT[action[0]])
#     state, reward, done, info = env.step(action)
#     # Show the game on the screen
#     img = env.render(mode='rgb_array')

#     # Display image with IPython; clear last iPython display on create
#     display(Image.fromarray(np.array(img), 'RGB'), clear=True)

In [ ]:
#####################################################################
# Testing the game
#####################################################################
# state = env.reset()
# state, reward, done, info = env.step([1])
# state, reward, done, info = env.step([1])
# state, reward, done, info = env.step([1])
# plt.figure(figsize=(15,12))
# for idx in range(state.shape[3]):
#     plt.subplot(1,4,idx+1)
#     plt.imshow(state[0][:,:,idx])
# plt.show()


# print(env.step(1)[1])
# print(state.shape)
# plt.imshow(state[0])
# plt.show()

# print(env.observation_space.shape)

In [ ]:
#####################################################################
# Testing the game
#####################################################################
# from IPython.display import clear_output, display
# from PIL import Image
# import numpy as np

# # Create a flag - restart or not
# done = True
# # Loop through each frame in the game
# for step in range(100000):
#     # Start the game to begin with
#     if done:
#         # Start the gamee
#         env.reset()
#     # Do random actions
#     state, reward, done, info = env.step(env.action_space.sample())
#     # Show the game on the screen
#     img = env.render(mode='rgb_array')

#     # Display image with IPython; clear last iPython display on create
#     display(Image.fromarray(np.array(img), 'RGB'), clear=True)

# # Close the game
# env.close()